# classification-10

## What's new:

1- https://chatgpt.com/c/690d9dcc-26cc-832f-8446-31080be617a7

## next step:

1- Improve labeling ( 3333 from 28-1111 )


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Input, Reshape, TimeDistributed, Lambda, RepeatVector
from tensorflow.keras import Input, layers, models, callbacks, metrics
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models, callbacks
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from sklearn.model_selection import train_test_split



In [ ]:
# 1- Load and Scaling Features

df = pd.read_csv('XAGUSD-197001010000--H1-rates.csv', sep='\t')
# Rename columns for easier access
df.rename(columns={
    '<DATE>': 'DATE',
    '<TIME>': 'TIME',
    '<OPEN>': 'OPEN',
    '<HIGH>': 'HIGH',
    '<LOW>': 'LOW',
    '<CLOSE>': 'CLOSE',
    '<TICKVOL>': 'TICKVOL',
    '<VOL>': 'VOL',
    '<SPREAD>': 'SPREAD'
}, inplace=True)

# ensure strings and strip any weird whitespace
df['DATE'] = df['DATE'].astype(str).str.strip()
df['TIME'] = df['TIME'].astype(str).str.strip()

df['DATETIME'] = pd.to_datetime(df['DATE'] + ' ' + df['TIME'], dayfirst=False, errors='coerce')
if df['DATETIME'].isna().any():
    raise ValueError("Some DATETIME values could not be parsed. Check date/time formats.")

# set DATETIME as index for reindexing
df = df.set_index('DATETIME').sort_index()

# --------------------------
# Create continuous hourly index & fill weekend gaps
# --------------------------
full_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq='h')

# Reindex to full hourly range so weekends/missing hours appear as NaN rows
df = df.reindex(full_index)

# Fill strategy:
# - Prices: forward-fill last known price across weekend gap (common approach for modeling continuity).
# - TICKVOL / VOL: set missing to 0 (no ticks during weekend).
# - SPREAD: forward-fill last known.
# Alternative: you could leave NaNs and drop sequences that cross weekends (safer but reduces data).
df[['OPEN', 'HIGH', 'LOW', 'CLOSE']] = df[['OPEN', 'HIGH', 'LOW', 'CLOSE']].ffill()
df['SPREAD'] = df['SPREAD'].ffill()
df['TICKVOL'] = df['TICKVOL'].fillna(0)
df['VOL'] = df['VOL'].fillna(0)

# Reset index to make DATETIME a regular column again
df = df.reset_index().rename(columns={'index': 'DATETIME'})

In [ ]:
df.shape

In [ ]:
import matplotlib.pyplot as plt

# Example: choose the start and end rows
start_row = 32200
end_row = 33000

# Select the range and make a copy to avoid SettingWithCopyWarning
subset = df.iloc[start_row:end_row + 1].copy()

# Ensure DATETIME is datetime type
subset['DATETIME'] = pd.to_datetime(subset['DATETIME'])

# Plot CLOSE price over time
plt.figure(figsize=(12, 6))
plt.plot(subset['DATETIME'], subset['CLOSE'], linewidth=1.0, color='blue')

# Labels and formatting
plt.title(f"Price Chart from Row {start_row} to {end_row}", fontsize=14)
plt.xlabel("Datetime", fontsize=12)
plt.ylabel("Close Price", fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


In [ ]:
# Specify how many rows to remove for model
nn = 33000  # Delete the first nn rows that do not follow the one-hour timeframe.
mm = 500  # Remove mm last row that the model should not see.

# Delete first nn and last mm rows
df_model = df.iloc[nn:len(df) - mm].reset_index(drop=True)

In [ ]:
def label_reversal_points(prices, window=6, threshold=0.0007):
    """
    Labels trend reversals (1=Buy, 2=Sell) based on local mean shifts.
    Smaller window & threshold increase sensitivity.
    """
    prices = np.asarray(prices)
    labels = [0] * len(prices)
    prev_trend = 0  # 1 = up, -1 = down, 0 = unknown

    for i in range(len(prices) - window):
        past = prices[i:i + window // 2]
        future = prices[i + window // 2:i + window]

        past_mean = np.mean(past)
        future_mean = np.mean(future)
        change = (future_mean - past_mean) / past_mean

        if change > threshold:
            curr_trend = 1  # Uptrend
        elif change < -threshold:
            curr_trend = -1  # Downtrend
        else:
            curr_trend = 0  # No significant trend

        # Detect reversal only when trend flips clearly
        if prev_trend == -1 and curr_trend == 1:
            labels[i + window // 2] = 1  # Buy
        elif prev_trend == 1 and curr_trend == -1:
            labels[i + window // 2] = 2  # Sell

        if curr_trend != 0:
            prev_trend = curr_trend

    return labels


df_model['Label'] = label_reversal_points(df_model['CLOSE'].values)

In [ ]:
print(df_model['Label'].value_counts().sort_index())  # 0, 1, 2

In [ ]:
import matplotlib.pyplot as plt


def plot_labeled_candles(df, n=190):
    """
    Plots the last n candles with BUY/SELL labels based on the 'Label' column.
    Assumes df already has a 'DATETIME' column.
    """
    # Drop NaN rows (e.g., weekend gaps)
    df_plot = df.dropna(subset=['CLOSE']).tail(n).copy()

    # Ensure DATETIME is a datetime column (optional safeguard)
    if not pd.api.types.is_datetime64_any_dtype(df_plot['DATETIME']):
        df_plot['DATETIME'] = pd.to_datetime(df_plot['DATETIME'])

    # === Plot Close Price ===
    plt.figure(figsize=(15, 6))
    plt.plot(df_plot['DATETIME'], df_plot['CLOSE'], label='Close Price', color='black', linewidth=1.5)

    # === Plot BUY (1) and SELL (2) signals ===
    for _, row in df_plot.iterrows():
        if row['Label'] == 1:  # BUY
            plt.axvline(x=row['DATETIME'], color='green', linestyle='--', linewidth=1)
            plt.text(row['DATETIME'], row['CLOSE'], 'BUY', color='green', ha='center', va='bottom', fontsize=9)
        elif row['Label'] == 2:  # SELL
            plt.axvline(x=row['DATETIME'], color='red', linestyle='--', linewidth=1)
            plt.text(row['DATETIME'], row['CLOSE'], 'SELL', color='red', ha='center', va='top', fontsize=9)

    # === Aesthetics ===
    plt.title(f'Last {n} Candles with Trend Reversal Labels')
    plt.xlabel('Datetime')
    plt.ylabel('Close Price')
    plt.xticks(rotation=45)
    plt.grid(True, linestyle='--', alpha=0.4)
    plt.tight_layout()
    plt.legend()
    plt.show()



In [ ]:
plot_labeled_candles(df_model)

In [ ]:
# --------------------------
# Imports & Hyperparams
# --------------------------

# Hyperparameters required by the user
WINDOW_SIZE = 60
FORECAST_HORIZON = 10
FEATURES = ['OPEN','HIGH','LOW','CLOSE','TICKVOL']
N_CLASSES = 3  # 0,1,2

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# --------------------------
# Assumptions: df_model exists
# --------------------------
# df_model must contain a DATETIME column (parseable), and Label column (0/1/2)
# and the feature columns defined above. DATETIME is continuous hourly series.
# Example: df_model['DATETIME'] dtype either datetime or string convertible to datetime.

# If DATETIME is string, convert once:
if not np.issubdtype(df_model['DATETIME'].dtype, np.datetime64):
    # try parsing common formats including your example "2025.07.24 11:00:00"
    df_model['DATETIME'] = pd.to_datetime(df_model['DATETIME'].astype(str).str.strip().str.replace('.', '-'),
                                          errors='coerce', dayfirst=False)
# Ensure sorted by DATETIME
df_model = df_model.sort_values('DATETIME').reset_index(drop=True)

# --------------------------
# Prepare supervised sequences
# --------------------------
X_list = []
y_list = []

n_rows = len(df_model)
max_start = n_rows - WINDOW_SIZE - FORECAST_HORIZON + 1
if max_start <= 0:
    raise ValueError("df_model is too short for WINDOW_SIZE + FORECAST_HORIZON")

for start in range(max_start):
    end = start + WINDOW_SIZE  # exclusive index for the window end
    X_window = df_model.loc[start:end-1, FEATURES].values  # shape (WINDOW_SIZE, n_features)
    y_window = df_model.loc[end:end+FORECAST_HORIZON-1, 'Label'].values  # length FORECAST_HORIZON
    # Only keep windows where the y_window has valid labels (no NaNs)
    if np.isnan(y_window).any():
        continue
    X_list.append(X_window)
    y_list.append(y_window.astype(int))

X = np.stack(X_list)  # shape (n_samples, WINDOW_SIZE, n_features)
y = np.stack(y_list)  # shape (n_samples, FORECAST_HORIZON)

print("Prepared data shapes:", X.shape, y.shape)

# Convert y to one-hot for training with categorical_crossentropy
y_onehot = np.array([to_categorical(sample, num_classes=N_CLASSES) for sample in y])  # (n_samples, H, C)

# --------------------------
# Train / Val split
# --------------------------
X_train, X_val, y_train, y_val = train_test_split(X, y_onehot, test_size=0.15, random_state=SEED, shuffle=True)
print("Train/Val shapes:", X_train.shape, X_val.shape, y_train.shape, y_val.shape)

# --------------------------
# Scaling features (fit on training only)
# --------------------------
# Fit scaler to training windows flattened (so scaler learns distribution per feature)
scaler = StandardScaler()
scaler.fit(X_train.reshape(-1, len(FEATURES)))  # (n_samples * WINDOW_SIZE, n_features)

def scale_windows(X_raw):
    n_samples = X_raw.shape[0]
    X_flat = X_raw.reshape(-1, len(FEATURES))
    X_scaled_flat = scaler.transform(X_flat)
    return X_scaled_flat.reshape(n_samples, WINDOW_SIZE, len(FEATURES))

X_train_scaled = scale_windows(X_train)
X_val_scaled = scale_windows(X_val)

# --------------------------
# Handle class imbalance (compute class weights then per-sample-per-timestep weights)
# --------------------------
# Flatten all target labels across train set to compute class frequency
y_train_flat = np.argmax(y_train, axis=2).reshape(-1)  # flatten
class_counts = np.bincount(y_train_flat, minlength=N_CLASSES).astype(float)
class_freq = class_counts / class_counts.sum()
# Inverse frequency (higher weight for rare classes)
class_weights = {i: (1.0 / freq if freq > 0 else 1.0) for i, freq in enumerate(class_freq)}
# Normalize weights to have mean 1
mean_w = np.mean(list(class_weights.values()))
class_weights = {k: (v / mean_w) for k, v in class_weights.items()}
print("class_counts:", class_counts, "class_weights(normalized mean=1):", class_weights)

# Build sample_weight matrix: shape (n_samples, FORECAST_HORIZON)
# Each timestep weight = class_weights[true_class]
def build_sample_weights(y_onehot, class_weights_map):
    y_idx = np.argmax(y_onehot, axis=2)  # (n_samples, H)
    weights = np.vectorize(class_weights_map.get)(y_idx)
    return weights.astype(np.float32)

sample_weight_train = build_sample_weights(y_train, class_weights)
sample_weight_val = build_sample_weights(y_val, class_weights)

# --------------------------
# Model: Encoder-Decoder LSTM -> TimeDistributed softmax
# --------------------------
n_features = len(FEATURES)

tf.keras.backend.clear_session()

encoder_inputs = layers.Input(shape=(WINDOW_SIZE, n_features), name='encoder_inputs')
x = layers.Conv1D(filters=64, kernel_size=3, padding='causal', activation='relu')(encoder_inputs)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=False))(x)  # encoder output
# Expand to decoder length
x = layers.RepeatVector(FORECAST_HORIZON)(x)
# Decoder
x = layers.LSTM(64, return_sequences=True)(x)
x = layers.TimeDistributed(layers.Dense(64, activation='relu'))(x)
decoder_outputs = layers.TimeDistributed(layers.Dense(N_CLASSES, activation='softmax'), name='decoder_outputs')(x)

model = models.Model(encoder_inputs, decoder_outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
# --------------------------
# Training callbacks
# --------------------------
es = callbacks.EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
mc = callbacks.ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, save_format='keras')
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-6)

# --------------------------
# Train model
# --------------------------
history = model.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val, sample_weight_val),
    epochs=60,
    batch_size=128,
    callbacks=[es, mc, reduce_lr],
    sample_weight=sample_weight_train,
    verbose=1
)

# After training you have a model saved at best_model.keras (native Keras format).
# You can also load with: model = tf.keras.models.load_model('best_model.keras')

In [ ]:
# --------------------------
# PREDICTION PIPELINE (for a single given_time)
# --------------------------
given_time = "2025.07.24 11:00:00"   # user-provided: note the space and dot-separated date
# Parse given_time into datetime (try to match user's format)
try:
    # transform "YYYY.MM.DD hh:mm:ss" to a parseable format
    gt = pd.to_datetime(given_time.replace('.', '-'), format=None, errors='coerce')
    if pd.isna(gt):
        # fallback generic parse
        gt = pd.to_datetime(given_time, errors='coerce')
except Exception:
    gt = pd.to_datetime(given_time, errors='coerce')

if pd.isna(gt):
    raise ValueError(f"given_time could not be parsed: {given_time}")

# Find the index of the row whose DATETIME equals given_time (we assumed DATETIME column matches)
match_idx = df_model.index[df_model['DATETIME'] == gt]
if len(match_idx) == 0:
    # if exact match not found, try to find the last row <= given_time
    match_idx = df_model.index[df_model['DATETIME'] <= gt]
    if len(match_idx) == 0:
        raise ValueError(f"No row at or before given_time {gt} found in df_model['DATETIME']")
    idx = match_idx[-1]
    # warn that we used the last available <= given_time
    print(f"Warning: exact given_time not found. Using last available index at {df_model.loc[idx,'DATETIME']}")
else:
    idx = match_idx[0]

# Determine start index for the 60-candle window (inclusive of idx)
start_idx = idx - WINDOW_SIZE + 1
if start_idx < 0:
    raise ValueError("Not enough history before given_time to form a full WINDOW_SIZE input")

X_input_raw = df_model.loc[start_idx:idx, FEATURES].values  # shape (WINDOW_SIZE, n_features)
if X_input_raw.shape[0] != WINDOW_SIZE:
    raise ValueError("Input window length mismatch (not equal to WINDOW_SIZE)")

# Scale using the previously-fitted scaler
X_input_scaled = scaler.transform(X_input_raw.reshape(-1, n_features)).reshape(1, WINDOW_SIZE, n_features)

# Model prediction
y_pred_prob = model.predict(X_input_scaled)  # shape (1, FORECAST_HORIZON, N_CLASSES)
y_pred_prob = y_pred_prob[0]  # (FORECAST_HORIZON, N_CLASSES)
y_pred_classes = np.argmax(y_pred_prob, axis=1)  # length FORECAST_HORIZON

# Enforce "realistic" behavior: user requested that most steps be 0.
# The model already learns imbalance, but we can optionally post-process:
# Optionally suppress very low-confidence non-zero predictions:
# If desired, uncomment the following lines to force a non-zero class only if prob > threshold
#threshold = 0.50
#for i in range(FORECAST_HORIZON):
#    top_prob = np.max(y_pred_prob[i])
#    top_class = np.argmax(y_pred_prob[i])
#    if top_class != 0 and top_prob < threshold:
#        y_pred_classes[i] = 0

# Build forecast datetimes: next 10 hourly steps immediately after given_time
first_forecast_dt = gt + pd.Timedelta(hours=1)
forecast_datetimes = pd.date_range(start=first_forecast_dt, periods=FORECAST_HORIZON, freq='H')

# Build predicted_df
predicted_df = pd.DataFrame({
    'DATETIME': forecast_datetimes,
    'forecast_class': y_pred_classes,
    'prob_0': y_pred_prob[:, 0],
    'prob_1': y_pred_prob[:, 1],
    'prob_2': y_pred_prob[:, 2],
})

# final line to show predicted_df (Jupyter will display)
predicted_df


# plot section

In [ ]:
# --------------------------
# === Visualization Block ===
# --------------------------
import pandas as pd
import sys

# --- 1. Historical window (last 4 real candles before forecast) ---
# --- Find the starting index ---
start_idx = df.index[df['DATETIME'] == pd.to_datetime(given_time)][0]

# --- 1. Extract next n candles ---
input_df = df.iloc[start_idx: start_idx + WINDOW_SIZE].copy()

historical_df = input_df.tail(4).copy()
historical_df

In [ ]:
# --- 2. Actual future 10 candles  ---
# Since input_df ends at index (start_idx - 1), actual_future_df starts right after that.
actual_future_start = start_idx
actual_future_end = start_idx + FORECAST_HORIZON
actual_future_df = df.iloc[actual_future_start - 1:actual_future_end].copy()
actual_future_df

In [ ]:
# --- 3. Create predicted_df (forecast for next 10 hours) ---
last_timestamp = input_df['DATETIME'].iloc[-1]
datetime_index = pd.date_range(
    start=last_timestamp + pd.Timedelta(hours=1),
    periods=FORECAST_HORIZON,
    freq='h'
)


# --- 4. Add text labels for clarity ---
predicted_df['label'] = predicted_df['forecast_class'].map({1: 'buy', 2: 'sell'}).fillna('')


# --- 5. Plot title & output settings ---
plot_title = 'Actual vs Predicted Forex Trend Reversals'
output_plot_path = None  # e.g., 'forecast_plot.png'



In [ ]:
# --- 6. Import your plotting utility ---
sys.path.insert(1, '../utils')
import forex_plot_utils_2

# --- 7. Plot all series ---
forex_plot_utils_2.plot_all_series(
    historical_df=historical_df,
    predicted_df=predicted_df,
    actual_future_df=actual_future_df,
    title=plot_title,
    output_path=output_plot_path
)


In [ ]:
# 11- Save Model

from datetime import datetime
import os
import pandas as pd
import matplotlib.pyplot as plt

# 11-1 Create timestamp and paths
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
model_filename = f'model_{timestamp}.keras'
model_path = os.path.join('saved_models', model_filename)

# 11-2 Directory to hold logs and extras
log_dir = os.path.join('saved_models', f'model_{timestamp}_logs')
os.makedirs(log_dir, exist_ok=True)

# 11-3 Save model
model.save(model_path)

# 11-4 Save training history
history_df = pd.DataFrame(history.history)
history_df.to_csv(os.path.join(log_dir, 'training_history.csv'), index=False)

# 11-5 Save training loss plot
plt.figure()
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(log_dir, 'training_loss.png'))
plt.close()

# 4. Evaluate on validation set (since no X_test/y_test defined)
final_train_loss = history.history['loss'][-1]
final_train_acc = history.history['cat_acc'][-1]
final_val_loss, final_val_acc = model.evaluate(X_val_scaled, y_val_cat, verbose=0)

# 5. Save model summary and performance metrics
summary_path = os.path.join(log_dir, 'model_log.txt')
with open(summary_path, 'w') as f:
    model.summary(print_fn=lambda x: f.write(x + '\n'))
    f.write('\n')
    f.write(f'Final Training Loss: {final_train_loss:.6f}\n')
    f.write(f'Final Training Accuracy: {final_train_acc:.6f}\n')
    f.write(f'Final Validation Loss: {final_val_loss:.6f}\n')
    f.write(f'Final Validation Accuracy: {final_val_acc:.6f}\n')

In [ ]:
model_path = 'saved_models/model_20251106_214146.keras'
model = keras.models.load_model(
    model_path,
    custom_objects={'loss_fn': focal_loss(), 'focal_loss': focal_loss()},
    safe_mode=False
)